**AI Final Project - Dental(tooth exaction) Question Answer**

Member:
- Chanon 63070503408
- Pechdanai 63070503434
- Fasai 63070503436

In [ ]:
!pip cache purge

Files removed: 0


In [ ]:
!pip install -U transformers
!pip install nlpaug

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 9.2 MB/s eta 0:00:00


In [ ]:
!pip install joblib

In [ ]:
from keras import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
from sklearn.model_selection import train_test_split

# Read and Preprocess data for classification

In [ ]:
# import data
df = pd.read_csv('qalist.csv')
df.describe()

FileNotFoundError: ignored

In [ ]:
df

In [ ]:
x = df['question'].values
y = df['class'].values

In [ ]:
# Assuming df is your DataFrame containing 'question' and 'class' columns
# Split the data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

x_train = train_df['question'].values
y_train = train_df['class'].values
x_test = test_df['question'].values
y_test = test_df['class'].values

In [ ]:
# Plot the target distributions
fig,ax =plt.subplots(1,2,figsize=(10,2))

bins = [0,1,2,3,4,5,6,7]

ax[0].hist(y_train,bins=bins)
ax[0].set_title('y train')
ax[1].hist(y_test,bins=bins)
ax[1].set_title('y test')

In [ ]:
# Tokenize the input text
tokenizer = AutoTokenizer.from_pretrained("timpal0l/mdeberta-v3-base-squad2", padding_side='left')

tokenizer_config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [ ]:
# Tokenize the input text and convert to NumPy array
x_train_tokenized = [tokenizer.encode(text, truncation=True, padding='max_length', max_length=20) for text in x_train]
x_train_tokenized = np.array([np.array(inner_list, dtype=np.int32) for inner_list in x_train_tokenized])

x_test_tokenized = [tokenizer.encode(text, truncation=True, padding='max_length', max_length=20) for text in x_test]
x_test_tokenized = np.array([np.array(inner_list, dtype=np.int32) for inner_list in x_test_tokenized])

# Flatten the tokenized sequences for decision tree
x_train_flat = x_train_tokenized.reshape(x_train_tokenized.shape[0], -1)
x_test_flat = x_test_tokenized.reshape(x_test_tokenized.shape[0], -1)


In [ ]:
x_tokenized = [tokenizer.encode(text, truncation=True, padding='max_length', max_length=20) for text in x]
x_tokenized = np.array([np.array(inner_list, dtype=np.int32) for inner_list in x_tokenized])
x_flat = x_tokenized.reshape(x_tokenized.shape[0], -1)


# Train classification with decision tree
we choose decision tree because it suitable with extra small dataset (our dataset have 203 samples and seperate to 7 classes)

In [ ]:
# Create a Decision Tree Classifier
best_params_augment = {'criterion': 'gini', 'max_depth': None, 'max_features': 'log2', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0}
best_params_nonaugment = {'criterion': 'gini', 'max_depth': 10, 'max_features': 'log2', 'max_leaf_nodes': 100, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 2, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0}
decision_tree = DecisionTreeClassifier(**best_params_nonaugment)

# Train the Decision Tree model
decision_tree.fit(x_train_flat, y_train)

# Make predictions on the test set
predictions = decision_tree.predict(x_test_flat)

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
print(f"Decision Tree Accuracy: {accuracy}")

f1 = f1_score(y_test, predictions, average= None)
average_f1 = mean(f1)
print(f"f1 Score: {average_f1}")

In [ ]:
def predict_class_for_sample(new_sample):
    # Tokenize the new sample
    new_sample_tokenized = tokenizer.encode(new_sample, truncation=True, padding='max_length', max_length=20)
    new_sample_tokenized = np.array(new_sample_tokenized, dtype=np.int32)

    # Flatten the tokenized sequence for decision tree
    new_sample_flat = new_sample_tokenized.flatten()

    # Reshape to (1, 50) since you are predicting a single sample
    new_sample_flat = new_sample_flat.reshape(1, -1)

    # Make prediction on the new sample
    prediction_new_sample = decision_tree.predict(new_sample_flat)

    label_to_text = ['ความหมาย','ประโยชน์','ข้อเสีย','ความเสี่ยง','การดูแลรักษา','ข้อควรระวัง','การเตรียมตัว','สาเหตุ']

    result = label_to_text[int(prediction_new_sample)]

    return result

# Train classification with Random Forest
we choose Random Forest because it suitable with extra small dataset (our dataset have 203 samples and seperate to 7 classes)

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split,GridSearchCV
# from sklearn.metrics import accuracy_score

# # Create a Random Forest classifier
# rf_model = RandomForestClassifier()

# # Define the parameter grid for grid search
# param_grid = {
#     'n_estimators': [50, 100, 200],
#     'criterion': ['gini', 'entropy'],
#     'max_depth': [None, 10, 20],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
#     'max_features': ['sqrt', 'log2'],
#     'min_weight_fraction_leaf': [0.0, 0.1, 0.2],
#     'max_leaf_nodes': [None, 50, 100],
#     'min_impurity_decrease': [0.0, 0.1, 0.2]
# }

# # Perform grid search to find the best combination of parameters
# grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3,n_jobs=-1)
# grid_search.fit(x_train_flat, y_train)

# # Get the best parameters from the grid search
# best_params = grid_search.best_params_
# print("Best Parameters:", best_params)

# # Train the model on the training data
# final_rf_model = RandomForestClassifier(**best_params)
# final_rf_model.fit(x_train_flat, y_train)

# # Make predictions on the test data
# rf_y_pred = final_rf_model.predict(x_test_flat)

# # Evaluate the accuracy of the model
# rf_accuracy = accuracy_score(y_test, rf_y_pred)
# print(f"Accuracy: {rf_accuracy}")

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split,GridSearchCV

# # Get the best parameters from the grid search
# rf_best_params_augment = {'criterion': 'gini', 'max_depth': None, 'max_features': 'log2', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 200}
# rf_best_params_nonaugment = {'criterion': 'gini', 'max_depth': 10, 'max_features': 'log2', 'max_leaf_nodes': 100, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 2, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 50}

# # Train the model on the training data
# final_rf_model = RandomForestClassifier(**rf_best_params_augment)
# final_rf_model.fit(x_train_flat, y_train)

# Evaluate the model

In [ ]:
from sklearn.metrics import f1_score, accuracy_score
from statistics import mean
# Make predictions on the test data
rf_y_pred = final_rf_model.predict(x_test_flat)

# Evaluate the accuracy of the model
rf_accuracy = accuracy_score(y_test, rf_y_pred)
print(f"Accuracy: {rf_accuracy}")

rf_f1 = f1_score(y_test, rf_y_pred, average= None)
rf_average_f1 = mean(rf_f1)
print(f"f1 Score: {rf_average_f1}")

NameError: ignored

# Load Random Forest Model

In [ ]:
from joblib import load

model_filename = 'random_forest_model.joblib'
final_rf_model = load(model_filename)

In [ ]:
def predict_class_for_sampleRF(new_sample):
    # Tokenize the new sample
    new_sample_tokenized = tokenizer.encode(new_sample, truncation=True, padding='max_length', max_length=20)
    new_sample_tokenized = np.array(new_sample_tokenized, dtype=np.int32)

    # Flatten the tokenized sequence for decision tree
    new_sample_flat = new_sample_tokenized.flatten()

    # Reshape to (1, 50) since you are predicting a single sample
    new_sample_flat = new_sample_flat.reshape(1, -1)

    # Make prediction on the new sample
    prediction_new_sample = final_rf_model.predict(new_sample_flat)

    label_to_text = ['ความหมาย','ประโยชน์','ข้อเสีย','ความเสี่ยง','การดูแลรักษา','ข้อควรระวัง','การเตรียมตัว','สาเหตุ']

    result = label_to_text[int(prediction_new_sample)]

    return result

# Load model from Huggingface via tranformers

In [ ]:
#define model
model = AutoModelForQuestionAnswering.from_pretrained("timpal0l/mdeberta-v3-base-squad2")

config.json:   0%|          | 0.00/879 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [ ]:
model

DebertaV2ForQuestionAnswering(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(251000, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True

# Test qa system by using question classifier

In [ ]:
df_question = pd.read_csv('questions.csv')
questions = df_question['question'].values
labels = df_question['class'].values

FileNotFoundError: ignored

In [ ]:
context_list = {
    "context1" : {
        "context" : "การถอนฟันคือกระบวนการทางทันตกรรมที่เกี่ยวข้องกับการนำฟันออกจากช่องปากของผู้ป่วยโดยใช้เครื่องมือทันตกรรมที่เหมาะสม การถอนฟันมีวัตถุประสงค์หลายประการเช่นเมื่อฟันที่คุณภาพด้อยและไม่สามารถฟื้นตัวได้เมื่อฟันเสียหายจากการบาดเจ็บหรือเมื่อมีปัญหาทันตกรรมอื่นๆที่ทำให้จำเป็นต้องถอนฟันเพื่อรักษาสุขภาพช่องปาก ในมุมมองทันตแพทย์การถอนฟันแสดงถึงกระบวนการที่เกี่ยวข้องกับการจัดการและรักษาฟันที่มีปัญหาซึ่งอาจเป็นส่วนหนึ่งของแผนรักษาโรคต่างๆ การถอนฟันอาจเกิดความปวดระหว่างกระบวนการและในช่วงเวลาการฟื้นตัว วัตถุประสงค์ของการถอนฟันอาจเป็นการรักษาสุขภาพช่องปากแก้ไขปัญหาทันตกรรมหรือจัดการกับฟันที่มีปัญหาเพื่อรักษาสุขภาพทันตกรรมโดยรวม",
        "label" : "ความหมาย"
        },
    "context2" : {
        "context" : "การถอนฟันมีประโยชน์มากมายทั้งในด้านการรักษาทันตกรรมและสุขภาพทั่วไป ประโยชน์ที่สำคัญที่สุดคือการแก้ไขปัญหาทันตกรรมเช่นการลดความปวดระหว่างฟันการป้องกันหรือรักษาโรคปริทันต์และการปรับปรุงสุขภาพช่องปากโดยรวม การถอนฟันยังช่วยลดโอกาสในการเกิดฟันผุและป้องกันการแพร่กระจายของเชื้อโรคในกรณีที่ฟันมีปัญหาร้ายแรง การถอนฟันมีความสำคัญในการป้องกันและรักษาภาวะทางระบบทั่วไปที่เกี่ยวข้องกับการรักษาทันตกรรมเช่นการลดความเสี่ยงของการติดเชื้อและการเกิดปัญหาที่เกี่ยวข้องกับฟัน การถอนฟันมีประโยชน์ทั้งในด้านทันตกรรมและสุขภาพทั่วไปของบุคคล การถอนฟันสามารถกำจัดอาการปวดจากสาเหตุต่างๆกำจัดเชื้อในช่องปากป้องกันการติดเชื้อในบริเวณใบหน้าช่วยให้ฟันแท้ขึ้นได้ตามปกติลดโอกาสเกิดฟันผุในฟันที่ขึ้นซ้อนเก",
        "label" : "ประโยชน์"
        },
     "context3" : {
        "context" : "ข้อเสียของการถอนฟันเช่นผู้ป่วยอาจมีความเจ็บปวดและเกิดความเครียดขณะรับการรักษา ในกรณีที่มีความเจ็บปวดมาก ทันตแพทย์จะให้ยาชาเฉพาะที่ระงับความเจ็บปวดเพิ่มเติม การถอนฟันมีข้อเสียและผลกระทบต่อสุขภาพทันตกรรมและทั่วไปของบุคคลได้มากมาย ข้อเสียของการถอนฟันอาจจะมีความเจ็บปวดและแผลที่เกิดขึ้นระหว่างกระบวนการและสูญเสีนฟันไปอาจทำให้เกิดความไม่มั่นใจหรือความลำบากในการเคี้ยวอาหารได้  การสูญเสียฟันยังสามารถทำให้การเคลื่อนไหวของฟันนี้ไม่สมดุลอาจทำให้เกิดปัญหาสุขภาพช่องปากและระบบทั่วไปเช่นการทำงานของกล้ามเนื้อในช่องปากเป็นต้น การถอนฟันอาจมีผลกระทบต่อการเคี้ยวอาหารความสวยงามการเรียงตัวของฟันการสบฟันเป็นต้น",
        "label" : "ข้อเสีย"
        },
     "context4" : {
        "context" : "การถอนฟันเป็นกระบวนการที่อาจมีความเสี่ยงต่อผู้ป่วยได้ หลังถอนฟันอาจมีอาการเจ็บปวดไม่สบายรวมไปถึงเกิดอาการบวมร่วมด้วย การถอนฟันยังเสี่ยงต่อการติดเชื้อในแผลหลังถอนฟันทำให้มีโอกาสเกิดความเสี่ยงแทรกซ้อนเช่นการติดเชื้อบวม การดูแลรักษาที่ถูกต้องหลังจากถอนฟันและปฏิบัติตามคำแนะนำของทันตแพทย์สามารถลดความเสี่ยงได้ การตรวจสอบสุขภาพทั่วไปก่อนการถอนฟันการปฏิบัติตามคำแนะนำของทันตแพทย์และการดูแลรักษาแผลให้ดีเป็นส่วนสำคัญในการลดความเสี่ยงที่เป็นไปได้ ความเสี่ยงที่มักพบคือการติดเชื้อบริเวณแผลถอนฟันหรือมีเลือดออกมาผิดปกติ อาการของการติดเชื้อหลังถอนฟันมักมีอากานปวดบวมแดงมีหนองเป็นไข้ หารมีอาการผิดปกติใดๆควรรีบไปพบทันตแพทย์ทันที หลังถอนฟันสามารถทานยาแก้ปวดเพื่อบรรเทาอาการปวดได้และควบคู่ไปกับการประคยเย็นบริเวณแก้มตรงที่มีแผล",
        "label" : "ความเสี่ยง"
        },
     "context5" : {
        "context" : "หลังถอนฟันบางคนอาจมีความเจ็บปวดบ้างในระหว่างหรือหลังจากกระบวนการ การดูแลแผลหลังถอนฟันสำคัญเพื่อป้องกันการติดเชื้อและส่งผลให้การฟื้นตัวดีขึ้น หลังถอนฟันแล้วสามารถแปรงฟันได้ปกติแต่ควรหลีกเลี่ยงการแปรงฟันในบริเวณแผลในช่วง24ชั่วโมงแรก การกินยาแก้ปวดที่แพทย์สั่งให้กินตามคำแนะนำจะช่วยควบคุมความปวดได้ควบคู่ไปกับการประคบเย็นบนแผลเป็นวิธีที่ดีเพื่อลดบวมและปวด หลังถอนฟันควรเว้นระยะเวลาในการกินอาหารแข็งหรือร้อนงดบ้วนปากแรงๆงบสูบบีหรี่และดื่มแอลกอฮอร์ การดูแลรักษาหลังถอนฟันมีคำแนะนำหลายอย่างเช่นไม่ควรดูดแผลและไม่บ้วนน้ำหรือน้ำลาย หลังถอนฟันควรการประคบเย็นในช่วงแรกและสามารถประคบร้อนได้หลังจากที่เลือดหยุดไหลและแผลเริ่มแห้ง หลังถอนฟันควรกัดผ่าก๊อซนาน1-2ชั่วโมงหรือจนกว่าเลือกจะหยุดไหล ควรรับประทานอาหารอ่อนๆในช่วงแรก สามารถทานยาแก้ปวดได้หากมีอาการปวดมาก โดยปกติยาชาจะออกฤทธิ์ประมาณ2-3ขึ้นอยู่กับปริมาณที่แพทย์เห็นสมควร ไหมละลายทั่วไปกะละลายหมดภายใน7-10วันและหากเป็นไหมปกติแพทย์จะนัดทำการตัดไหมได้หลังการถอนฟัน5-7วัน โดยปกติแล้วหลังถอนฟันเลือดจะไหลประมาณ1-2ชั่วโมงและมีเลือกซึมบ้างใน24ชั่วโมงแรกและจะค่อยๆหยุดลง",
        "label" : "การดูแลรักษา"
        },
     "context6" : {
        "context" : "หลังถอนฟันมีข้อควรระวังและข้อห้ามที่ควรคำนึงถึงหลายอย่างเช่นเลี่ยงการทานอาหารร้อนหรือเผ็ดควรทานอาหารอ่อนๆไม่ควรสูลลุหรี่หรือดื่มแอลกอฮอร์อีกทั้งยังควรระวังเวลาแปรงฟันหรือทำความสะอาดช่องปากอีกด้วย หลังถอนฟันควรหลีกเลี่ยงการกินอาหารแข็งร้อนหรือเผ็ดเพื่อป้องกันการกระทบต่อแผลที่เกิดขึ้น สามารถการแปรงฟันได้ปกติแต่คสรระวังบีิเวณที่เป็นแผลจากการถอนฟันเป็นพิเศษเพื่อป้องกันการเพิ่มเจ็บปวดและบวม. หลังถอนฟันไม่ควรกินอาหารร้อนหรือเผ็ดมากๆ ไม่ควรดื่มน้ำร้อนหลังถอนฟันควรตื่มเป้นน้ำอุณหภูมิห้อง  การกินอาหารที่ร้อนหรือเผ็ดอาจไปกระตุ้นแผลและการเพิ่มความเจ็บปวดได้ หลังทำหารถอนฟันควรกัดผ้าก๊อชไว้1-2ชั่วโมงหรือจนกว่าเลือกจะหยุด",
        "label" : "ข้อควรระวัง"
        },
     "context7" : {
        "context" : "ก่อนถอนฟันควรปรึกษากับทันตแพทย์เพื่อประเมินสภาพทางทันตกรรมของผู้ป่วยทำรายการตรวจสุขภาพทันตกรรมเพื่อตรวจสอบปัญหาทางทันตกรรมที่อาจเกี่ยวข้องและพักผ่อนให้เพียงพอ ก่อนถอนฟันควรสามารถรับประทานอาหารหรือเครื่องดื่มได้พอประมาณแต่ควรเว้นระยะเวลาให้เหมาะสม ก่อนถอนฟันควรพักผ่อนมาให้เรียบร้อยงดสูบบุหรี่และงดเครื่องดื่มที่มีแอลกอฮอร์ สำหรับผู้ที่มีโรงประจำตัวสามารถรับประทานยาประจำได้ตามปกติทั้งนี้ทั้งนั้นควรปรึกษาแพทย์ก่อน",
        "label" : "การเตรียมตัว"
        },
     "context8" : {
        "context" : "การถอนฟันเป็นกระบวนการที่ทำเมื่อฟันมีปัญหาหรือความเสียหายมากขึ้นและไม่สามารถรักษาได้โดยวิธีอื่นๆ ปัญหาที่อาจทำให้ต้องถอนฟันเช่นฟันผุที่มีความเสียหายมากฟันอยู่ในตำแหน่งที่ไม่ถูกต้องฟันหักหรือแตกจากการเกิดอุบัติเหตุ  อาการที่บ่งบอกว่าต้องถอนฟันอาจมีการปวดฟันรุนแรง การตรวจรักษาโดยทันตแพทย์จะช่วยประเมินว่าฟันมีปัญหาอะไรบ้างและวิธีการรักษาที่เหมาะสม ในบางกรณีการถอนฟันจำเป็นเพื่อรักษาปัญหาทันตกรรมอย่างถาวรและป้องกันการแพร่กระจายของปัญหา การถอนฟันมีความจำเป็นหรือไม่ขึ้นอยู่กับสถานการณ์และความรุนแรงของปัญหาทันตกรรมในแต่ละกรณี ไม่ควรถอนฟันเองเนื่องจากการถอนฟันเองโดยใช้มือหรือใช้เครื่องมืออื่นๆที่ไม่ใช่อุปกรณ์ทางทันตกรรมโดยเฉพาะจะส่งผลให้ฟันเสี่ยงติดเชื้อได้ง่าย",
        "label" : "สาเหตุ"
        }
    }

In [ ]:
def compare_labels(context_list, question_label):
    for key, value in context_list.items():
        context = value["context"]
        context_label = value["label"]

        # Compare labels
        if context_label == question_label:
            flag = True
            matched_context = {
                "flag": True,
                "key": key,
                "context": context,
                "label": context_label
            }
            return flag, matched_context
        else:
            flag = False
            matched_context = {
                "key": "",
                "context": "",
                "label": ""
            }
    return flag, matched_context

In [ ]:
nlp = pipeline('question-answering', model=model, tokenizer=tokenizer)
def answer(question, context):
  nlp_output = nlp({
    'question': question,
    'context': context
  })
  return nlp_output['answer'], nlp_output['score']

In [ ]:
#Test qa system with 1 question from user input
input_question = input('Enter your question : ')
q_label = predict_class_for_sampleRF(input_question)
ismatched, context_question = compare_labels(context_list, q_label)
key_context_result = context_question['label']
if ismatched == True:
  context_q = context_question['context']
  answer_result, ans_score  = answer(input_question, context_q)
  print(f"answer : {answer_result}")
  print(f"label : {key_context_result}")
  print(f"score : {ans_score}")
else:
  print(f"input question does not match")

Enter your question : ทอนฟันคืออะไร
answer :  การถอนฟันสามารถกำจัดอาการปวดจากสาเหตุต่างๆกำจัดเชื้อในช่องปากป้องกันการติดเชื้อในบริเวณใบหน้าช่วยให้ฟันแท้ขึ้นได้ตามปกติลดโอกาสเกิดฟันผุในฟันที่ขึ้นซ้อนเก
label : ประโยชน์
score : 0.028763260692358017


#mean f1 score for all question in qalist.csv

In [ ]:
import time

start_time = time.time()

ans_scores = []
for question in questions:
  input_question = question
  q_label = predict_class_for_sampleRF(input_question)
  ismatched, context_question = compare_labels(context_list, q_label)
  key_context_result = context_question['label']
  if ismatched == True:
    context_q = context_question['context']
    answer_result, ans_score  = answer(input_question, context_q)
    ans_scores.append(ans_score)
  else:
    print(f"input question does not match")

end_time = time.time()
execution_time = end_time - start_time

Execution time: 181.0592484474182 seconds
Mean Score: 0.05207948724920095


In [ ]:
print(f"Execution time: {execution_time} seconds")
mean_score = np.mean(ans_scores)
print(f"Mean Score: {mean_score}")

Execution time: 181.0592484474182 seconds
Mean Score: 0.05207948724920095


# Test qa system without using question classifier

In [ ]:
all_context = ""
for key, value in context_list.items():
    context = value["context"]

    all_context = all_context + context

all_context

'การถอนฟันคือกระบวนการทางทันตกรรมที่เกี่ยวข้องกับการนำฟันออกจากช่องปากของผู้ป่วยโดยใช้เครื่องมือทันตกรรมที่เหมาะสม การถอนฟันมีวัตถุประสงค์หลายประการเช่นเมื่อฟันที่คุณภาพด้อยและไม่สามารถฟื้นตัวได้เมื่อฟันเสียหายจากการบาดเจ็บหรือเมื่อมีปัญหาทันตกรรมอื่นๆที่ทำให้จำเป็นต้องถอนฟันเพื่อรักษาสุขภาพช่องปาก ในมุมมองทันตแพทย์การถอนฟันแสดงถึงกระบวนการที่เกี่ยวข้องกับการจัดการและรักษาฟันที่มีปัญหาซึ่งอาจเป็นส่วนหนึ่งของแผนรักษาโรคต่างๆ การถอนฟันอาจเกิดความปวดระหว่างกระบวนการและในช่วงเวลาการฟื้นตัว วัตถุประสงค์ของการถอนฟันอาจเป็นการรักษาสุขภาพช่องปากแก้ไขปัญหาทันตกรรมหรือจัดการกับฟันที่มีปัญหาเพื่อรักษาสุขภาพทันตกรรมโดยรวมการถอนฟันมีประโยชน์มากมายทั้งในด้านการรักษาทันตกรรมและสุขภาพทั่วไป ประโยชน์ที่สำคัญที่สุดคือการแก้ไขปัญหาทันตกรรมเช่นการลดความปวดระหว่างฟันการป้องกันหรือรักษาโรคปริทันต์และการปรับปรุงสุขภาพช่องปากโดยรวม การถอนฟันยังช่วยลดโอกาสในการเกิดฟันผุและป้องกันการแพร่กระจายของเชื้อโรคในกรณีที่ฟันมีปัญหาร้ายแรง การถอนฟันมีความสำคัญในการป้องกันและรักษาภาวะทางระบบทั่วไปที่เกี่ยวข้องกับการรักษาท

# mean f1 score for all question (no classfier) qalist.csv

In [ ]:
import time

start_time_NOClassifier = time.time()

ans_scores2 = []
for question in questions:
  input_question = question
  answer_result, ans_score  = answer(input_question, all_context)
  ans_scores2.append(ans_score)

end_time_NOClassifier  = time.time()
execution_time_NOClassifier  = end_time - start_time

In [ ]:
print(f"Execution time: {execution_time_NOClassifier} seconds")
mean_score2 = np.mean(ans_scores2)
print(f"Mean Score: {mean_score2}")